In [ ]:
import os

os.environ['HF_HUB_DISABLE_IMPLICIT_TOKEN'] = '1'
os.environ['WANDB_DISABLED'] = 'true'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

print("Installing transformers version so no 404 error occurs...")
!pip uninstall -y peft -q
!pip install -q transformers==4.40.1 accelerate==0.27.0
print("[SUCCESS]: Environment variables have been set")

In [ ]:
import torch

print("[CHECKING]: GPU")

if torch.cuda.is_available():
    print(f"[SUCCESS]: Available GPU --> {torch.cuda.get_device_name(0)}")
    print(f"[SUCCESS]: GPU Memory --> {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("[ERROR]: GPU was not detected. Please enable it to --> GPU T4 x 2")
    